# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 27 2022 9:53AM,253810,10,CTF0011562,"Citieffe, Inc.",Released
1,Dec 27 2022 9:52AM,253809,12,HH8610,Hush Hush,Released
2,Dec 27 2022 9:51AM,253808,10,CTF0011561,"Citieffe, Inc.",Released
3,Dec 27 2022 9:32AM,253806,16,TASA-284192,TASA USA Inc.,Released
4,Dec 27 2022 9:19AM,253801,12,HH-38953,Hush Hush,Released
5,Dec 27 2022 9:19AM,253801,12,HH-38955,Hush Hush,Released
6,Dec 27 2022 9:19AM,253801,12,HH-38956,Hush Hush,Released
7,Dec 27 2022 9:19AM,253801,12,HH-38957,Hush Hush,Released
8,Dec 27 2022 9:19AM,253801,12,HH-38958,Hush Hush,Released
9,Dec 27 2022 9:19AM,253801,12,HH-38959,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,253805,Released,1
15,253806,Released,1
16,253808,Released,1
17,253809,Released,1
18,253810,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253805,NaN,1.0
253806,NaN,1.0
253808,NaN,1.0
253809,NaN,1.0
253810,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253783,0.0,1.0
253784,17.0,12.0
253788,0.0,1.0
253789,0.0,1.0
253790,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253783,0,1
253784,17,12
253788,0,1
253789,0,1
253790,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253783,0,1
1,253784,17,12
2,253788,0,1
3,253789,0,1
4,253790,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253783,,1
1,253784,17,12
2,253788,,1
3,253789,,1
4,253790,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 27 2022 9:53AM,253810,10,"Citieffe, Inc."
1,Dec 27 2022 9:52AM,253809,12,Hush Hush
2,Dec 27 2022 9:51AM,253808,10,"Citieffe, Inc."
3,Dec 27 2022 9:32AM,253806,16,TASA USA Inc.
4,Dec 27 2022 9:19AM,253801,12,Hush Hush
42,Dec 27 2022 9:17AM,253805,19,ACG North America LLC
43,Dec 27 2022 9:13AM,253804,10,"Senseonics, Incorporated"
44,Dec 27 2022 9:11AM,253798,10,ISDIN Corporation
84,Dec 27 2022 9:03AM,253799,10,Bio-PRF
85,Dec 27 2022 9:03AM,253795,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 27 2022 9:53AM,253810,10,"Citieffe, Inc.",,1
1,Dec 27 2022 9:52AM,253809,12,Hush Hush,,1
2,Dec 27 2022 9:51AM,253808,10,"Citieffe, Inc.",,1
3,Dec 27 2022 9:32AM,253806,16,TASA USA Inc.,,1
4,Dec 27 2022 9:19AM,253801,12,Hush Hush,,38
5,Dec 27 2022 9:17AM,253805,19,ACG North America LLC,,1
6,Dec 27 2022 9:13AM,253804,10,"Senseonics, Incorporated",,1
7,Dec 27 2022 9:11AM,253798,10,ISDIN Corporation,,40
8,Dec 27 2022 9:03AM,253799,10,Bio-PRF,,1
9,Dec 27 2022 9:03AM,253795,10,ISDIN Corporation,,61


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 9:53AM,253810,10,"Citieffe, Inc.",1,
1,Dec 27 2022 9:52AM,253809,12,Hush Hush,1,
2,Dec 27 2022 9:51AM,253808,10,"Citieffe, Inc.",1,
3,Dec 27 2022 9:32AM,253806,16,TASA USA Inc.,1,
4,Dec 27 2022 9:19AM,253801,12,Hush Hush,38,
5,Dec 27 2022 9:17AM,253805,19,ACG North America LLC,1,
6,Dec 27 2022 9:13AM,253804,10,"Senseonics, Incorporated",1,
7,Dec 27 2022 9:11AM,253798,10,ISDIN Corporation,40,
8,Dec 27 2022 9:03AM,253799,10,Bio-PRF,1,
9,Dec 27 2022 9:03AM,253795,10,ISDIN Corporation,61,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 9:53AM,253810,10,"Citieffe, Inc.",1,
1,Dec 27 2022 9:52AM,253809,12,Hush Hush,1,
2,Dec 27 2022 9:51AM,253808,10,"Citieffe, Inc.",1,
3,Dec 27 2022 9:32AM,253806,16,TASA USA Inc.,1,
4,Dec 27 2022 9:19AM,253801,12,Hush Hush,38,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 9:53AM,253810,10,"Citieffe, Inc.",1,NaN
1,Dec 27 2022 9:52AM,253809,12,Hush Hush,1,NaN
2,Dec 27 2022 9:51AM,253808,10,"Citieffe, Inc.",1,NaN
3,Dec 27 2022 9:32AM,253806,16,TASA USA Inc.,1,NaN
4,Dec 27 2022 9:19AM,253801,12,Hush Hush,38,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 27 2022 9:53AM,253810,10,"Citieffe, Inc.",1,0.0
1,Dec 27 2022 9:52AM,253809,12,Hush Hush,1,0.0
2,Dec 27 2022 9:51AM,253808,10,"Citieffe, Inc.",1,0.0
3,Dec 27 2022 9:32AM,253806,16,TASA USA Inc.,1,0.0
4,Dec 27 2022 9:19AM,253801,12,Hush Hush,38,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2791771,131,0.0
12,761407,40,0.0
16,253806,1,0.0
19,507588,2,0.0
20,253784,12,17.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2791771,131,0.0
1,12,761407,40,0.0
2,16,253806,1,0.0
3,19,507588,2,0.0
4,20,253784,12,17.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,131,0.0
1,12,40,0.0
2,16,1,0.0
3,19,2,0.0
4,20,12,17.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,131.0
1,12,Released,40.0
2,16,Released,1.0
3,19,Released,2.0
4,20,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20
Status,,,,,
Executing,0.0,0.0,0.0,0.0,17.0
Released,131.0,40.0,1.0,2.0,12.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20
0,Executing,0.0,0.0,0.0,0.0,17.0
1,Released,131.0,40.0,1.0,2.0,12.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20
0,Executing,0.0,0.0,0.0,0.0,17.0
1,Released,131.0,40.0,1.0,2.0,12.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()